# OASIS Data Collection
 - Demand
 - Market Prices
 - Generation

## Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

In [2]:
# --- CAISO OASIS → Pandas → Plot (single Jupyter cell) ---
# If needed (first run): 
# %pip install pandas matplotlib requests python-dateutil pytz

import io, zipfile, requests, pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil import tz

# ===================== USER INPUT =====================
REPORT = "RTM_5MIN"   # options: "RTM_5MIN", "DAM_HOURLY", "RTPD_15MIN"
NODES  = ["NP15_GEN-APND", "SP15_GEN-APND"]  # comma-separated list of hubs/nodes
START  = "2025-08-01"  # inclusive, YYYY-MM-DD (Pacific calendar date)
END    = "2025-08-03"  # inclusive, YYYY-MM-DD (Pacific calendar date)
# ======================================================

# Map the simple report names to OASIS parameters
REPORT_MAP = {
    "RTM_5MIN":   dict(queryname="PRC_INTVL_LMP", market_run_id="RTM",  version="2"),
    "DAM_HOURLY": dict(queryname="PRC_LMP",       market_run_id="DAM",  version="12"),
    "RTPD_15MIN": dict(queryname="PRC_RTPD_LMP",  market_run_id="RTPD", version="7"),
}

BASE = "https://oasis.caiso.com/oasisapi/SingleZip"
params_core = REPORT_MAP[REPORT]

# Convert user Pacific dates to UTC day windows expected by OASIS
PT = tz.gettz("America/Los_Angeles")
UTC = tz.gettz("UTC")
start_pt = datetime.fromisoformat(START).replace(tzinfo=PT).astimezone(UTC)
end_pt   = (datetime.fromisoformat(END) + timedelta(days=1)).replace(tzinfo=PT).astimezone(UTC)

def fmt(dt_utc):
    # OASIS wants e.g. 20230402T07:00-0000 (always UTC with -0000)
    return dt_utc.strftime("%Y%m%dT%H:%M-0000")

def fetch_oasis_day(day_utc_start, day_utc_end, nodes):
    """Fetch one UTC day window and return a tidy DataFrame or empty DataFrame."""
    p = dict(
        queryname=params_core["queryname"],
        market_run_id=params_core["market_run_id"],
        resultformat="6",
        version=params_core["version"],
        startdatetime=fmt(day_utc_start),
        enddatetime=fmt(day_utc_end),
        node=",".join(nodes)
    )
    r = requests.get(BASE, params=p, timeout=90)
    r.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        csvs = [n for n in z.namelist() if n.endswith(".csv")]
        if not csvs:
            return pd.DataFrame()
        df = pd.read_csv(z.open(csvs[0]))
    if df.empty:
        return df

    # Normalize schema
    ts_col = "INTERVALSTARTGMT" if "INTERVALSTARTGMT" in df.columns else (
             "INTERVALSTART_GMT" if "INTERVALSTART_GMT" in df.columns else None)
    node_col = "NODE" if "NODE" in df.columns else ("PRC_NODE_ID" if "PRC_NODE_ID" in df.columns else None)

    # Minimal set of columns for pivot
    candidate_cols = [ts_col, node_col, "LMP_TYPE", "VALUE"]
    have_cols = [c for c in candidate_cols if c and c in df.columns]
    df = df[have_cols].rename(columns={ts_col: "interval_gmt", node_col: "node", "VALUE": "value"})

    # Pivot LMP components into columns (lmp/energy/cong/loss/ghg where available)
    if "LMP_TYPE" in df.columns:
        df = df.pivot_table(index=["interval_gmt", "node"], columns="LMP_TYPE", values="value", aggfunc="first").reset_index()
        df.columns = [str(c).lower() for c in df.columns]
        for c in ["lmp", "energy", "cong", "loss", "ghg"]:
            if c not in df.columns:
                df[c] = pd.NA
    else:
        df.columns = [c.lower() for c in df.columns]
        if "lmp" not in df.columns and "value" in df.columns:
            df = df.rename(columns={"value":"lmp"})
        for c in ["energy","cong","loss","ghg"]:
            if c not in df.columns:
                df[c] = pd.NA

    # Convert to timezone-aware and then to Pacific for plotting convenience
    df["interval_gmt"] = pd.to_datetime(df["interval_gmt"], utc=True, errors="coerce")
    df = df.dropna(subset=["interval_gmt"])
    df["time_pt"] = df["interval_gmt"].dt.tz_convert(PT)
    return df[["time_pt", "node", "lmp", "energy", "cong", "loss", "ghg"]]

# Iterate day-by-day (OASIS prefers daily windows for price reports)
frames = []
cursor = start_pt.replace(hour=0, minute=0, second=0, microsecond=0)
while cursor < end_pt:
    nxt = cursor + timedelta(days=1)
    day_df = fetch_oasis_day(cursor, nxt, NODES)
    if not day_df.empty:
        frames.append(day_df)
    cursor = nxt

prices = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(
    columns=["time_pt","node","lmp","energy","cong","loss","ghg"]
)

display(prices.head(10))
print(f"Rows: {len(prices):,}")

# -------- Plot --------
if not prices.empty:
    fig, ax = plt.subplots(figsize=(11, 5))
    for node, sub in prices.groupby("node"):
        ax.plot(sub["time_pt"], sub["lmp"], label=node, linewidth=1.2)
    title_map = {
        "RTM_5MIN": "CAISO Real-Time (5-min) LMP",
        "DAM_HOURLY": "CAISO Day-Ahead (Hourly) LMP",
        "RTPD_15MIN": "CAISO Real-Time Pre-Dispatch (15-min) LMP"
    }
    ax.set_title(f"{title_map[REPORT]} — {START} to {END} PT")
    ax.set_xlabel("Time (Pacific)")
    ax.set_ylabel("Price ($/MWh)")
    ax.grid(True, alpha=0.3)
    ax.legend(loc="upper right")
    plt.tight_layout()
    plt.show()
else:
    print("No data returned. Try a shorter date range, different nodes, or a different REPORT mapping.")


HTTPError: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_INTVL_LMP&market_run_id=RTM&resultformat=6&version=2&startdatetime=20250802T00%3A00-0000&enddatetime=20250803T00%3A00-0000&node=NP15_GEN-APND%2CSP15_GEN-APND